#4 Class-Classification
## Natural Image
## first model  >>  baseline_resnet50


In [ ]:
import numpy as np
import urllib.request
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
img_rows, img_cols = 224, 224 #number of rows and columns to convert the images to
input_shape = (img_rows, img_cols, 3)#format to store the images (rows, columns,channels) called channels last

In [ ]:
def url_to_image(url):
	# download the image, convert it to a NumPy array, and then read
	# it into OpenCV format
	resp = urllib.request.urlopen(url)
	image = np.asarray(bytearray(resp.read()), dtype="uint8")
	image = cv2.imdecode(image, cv2.IMREAD_COLOR)
 
	# return the image
	return image

In [ ]:
#%clear base_model

import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

#train_datagen = ImageDataGenerator(rescale=1./255)
#valid_datagen = ImageDataGenerator(rescale=1./255)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)   
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

train_generator1 = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/Dataset/Train/',
        classes=['airplane','motorbike','car','person'],
        target_size=(img_rows, img_cols),batch_size=32,class_mode='categorical')

valid_generator1 = valid_datagen.flow_from_directory(
        '/content/drive/MyDrive/Dataset/Validate/',
        classes=['airplane','motorbike','car','person'],
        target_size=(img_rows, img_cols),batch_size=32,class_mode='categorical')


Found 1532 images belonging to 4 classes.
Found 707 images belonging to 4 classes.


In [ ]:
base_model1 = tf.keras.applications.ResNet50(weights='imagenet', include_top = False)

In [ ]:
for layer in base_model1.layers:   layer.trainable = False

In [ ]:
x = base_model1.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)

x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
preds = tf.keras.layers.Dense(4, activation ='softmax')(x)

model1 = tf.keras.models.Model(inputs=base_model1.input, outputs=preds)


In [ ]:
model1.compile(optimizer='RMSProp', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model1.fit(
      train_generator1,
      steps_per_epoch=train_generator1.n//train_generator1.batch_size,
      epochs=4,
      validation_data=valid_generator1,
      validation_steps=25)

Epoch 1/4
47/47 [==============================] - 380s 8s/step - loss: 1.2428 - accuracy: 0.8967 - val_loss: 4.3823e-04 - val_accuracy: 1.0000
Epoch 2/4
47/47 [==============================] - 233s 5s/step - loss: 8.0005e-06 - accuracy: 1.0000
Epoch 3/4
47/47 [==============================] - 233s 5s/step - loss: 7.0753e-07 - accuracy: 1.0000
Epoch 4/4
47/47 [==============================] - 233s 5s/step - loss: 4.5379e-08 - accuracy: 1.0000


In [ ]:
import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator #, array_to_img, img_to_array, load_img

model1.save('/content/drive/My Drive/db/m1.h5') 

'''
model1 = tf.keras.models.load_model('m1.h5')

'''

"\nmodel1 = tf.keras.models.load_model('m1.h5')\n\n"

In [ ]:
# ntest_total = 745, accuracy_% = 0.9168
# classified = 683
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)
#test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = valid_datagen.flow_from_directory(
        '/content/drive/MyDrive/Dataset/Test',
        classes=['airplane','motorbike','car','person'],
        target_size=(img_rows, img_cols),batch_size=32,class_mode='categorical')

Found 669 images belonging to 4 classes.


In [ ]:
history=model1.evaluate(test_generator)

21/21 [==============================] - 113s 5s/step - loss: 0.0312 - accuracy: 0.9985
